# 2020 Stock Market Crash and Rebound Amid Coronavirus
---

In this tutorial, we will demonstrate the usage of market indexes.

In [1]:
import pandas as pd

import capon
from capon.datasets import load_stock_indexes

ImportError: cannot import name 'load_metadata' from 'capon.datasets.metadata' (/Users/eyalgal/Dropbox/PhD/Code/capon/capon/datasets/metadata.py)

## Market Indexes Historical Data

First, we need the tick symbols of the required index(es).

If we don't know it, we can obviously use Yahoo's [Major World Indices](https://finance.yahoo.com/world-indices/) page. Or, alternatively, we can get it from `capon`'s datasets using `load_stock_indexes()`.

In [ ]:
stock_indexes = load_stock_indexes()
display(stock_indexes)

Now, we can fetch hisotrical data for the relevant indexes using `capon.stock(symbol)`.

In [ ]:
key_indexes = ['^GSPC', '^DJI', '^IXIC', '^RUT']

In [ ]:
indexes = pd.concat([capon.stock(tick, range='1y', interval='1d') 
                    for tick in key_indexes], sort=False)
indexes = indexes[pd.to_datetime(indexes['timestamp'], utc=True).dt.year == 2020]
indexes

That's it! We have our data.

In [ ]:
import plotly.express as px
px.defaults.template = 'none'

px.line(indexes, x='timestamp', y='adjclose', color='symbol')

## Coronavirus Crash and Rebound

To quantify the crash and the level of rebound, let us normalize all indexes to their status prior to the Coronavirus crash. Specifically, we will use the values from 2020-02-19, which was the date of the S&P all time high (prior to the crash).

For the sake of `pandas` plotting we should transform the data into a wide form

In [ ]:
indexes_ts = indexes \
    .assign(date=lambda df: pd.to_datetime(df['timestamp'].apply(lambda dt: dt.tz_localize(None).date()))) \
    .pivot_table(index='date', columns=['symbol'], values='adjclose') \
    .rename(stock_indexes.set_index('symbol')['name'], axis=1) \
    .loc['2020-01-01':]
indexes_ts.head()

The S&P all time high will be used as the anchor for normalization.

In [ ]:
anchor = indexes_ts['S&P 500'].loc[:'2020-04-01'].idxmax()
anchor

Normalizing index values to their value at anchor time, we get

In [ ]:
anchor_values = indexes_ts.bfill().loc[anchor:].iloc[0]
indexes_ts_normed = (indexes_ts/anchor_values).dropna()

indexes_ts_normed.tail(5) \
    .style.format('{:+.2%}').background_gradient(axis=None)

In [ ]:
fig = px.line(indexes_ts_normed.stack().to_frame('value').reset_index(),
              x='date', y='value', color='symbol')
fig.update_layout(yaxis_tickformat='%')
fig.update_layout(shapes=[
    dict(
      type='line',
      yref='paper', y0=0, y1=1, 
      xref='x', x0=anchor, x1=anchor,
      line=dict(width=1.5, dash='dash', color='rgba(0,0,0,0.5)'),
    )
])

We can see that the NASDAQ had the fastest recovery, and on 2020-06-10 even exceeded its prior value. Actually, Nasdaq hitted 10,000 for the first time ever!